In [2]:
import pandas as pd
import numpy as np
import torch
from numba import jit

In [4]:
stock=pd.read_csv('weekly_stock.csv',low_memory=False)

In [6]:
stock.head()

,code,name,time,mv,pb,return,volume
0,000001.SZ,平安银行,2006-01-06,1.247272e+10,2.4871,0.043974,54697053.0
1,000001.SZ,平安银行,2006-01-13,1.208356e+10,2.4095,-0.031201,56915328.0
2,000001.SZ,平安银行,2006-01-20,1.212247e+10,2.4173,0.003221,36125086.0
3,000001.SZ,平安银行,2006-01-25,1.235597e+10,2.4638,0.019262,27011132.0
4,000001.SZ,平安银行,2006-02-10,1.208356e+10,2.4095,-0.022047,43678212.0


In [8]:
stock[['return','pb']]=stock[['return','pb']].apply(pd.to_numeric,errors='coerce')
stock['return']=stock['return']*0.01
stock.sort_values(['code','time'],inplace=True)

In [9]:
stock.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2042460 entries, 0 to 2042459
Data columns (total 7 columns):
 #   Column  Dtype  
---  ------  -----  
 0   code    object 
 1   name    object 
 2   time    object 
 3   mv      float64
 4   pb      float64
 5   return  float64
 6   volume  float64
dtypes: float64(4), object(3)
memory usage: 124.7+ MB


In [24]:
# lag一期
shift=[]
for i,data in stock.groupby('code'):
    shift.append(data.loc[:,['mv','pb','return','volume']].shift(1))
shift=pd.concat(shift,axis=0)
old=stock.loc[:,'code':'time']
stock_new=pd.concat([old,shift],axis=1)

stock_new.dropna(how='any',inplace=True)

In [25]:
stock_new['bm']=1/stock_new['pb']
stock_new.drop(columns='pb',inplace=True)

In [26]:
stock_new.reset_index(drop=True,inplace=True)

In [27]:
# 因子排序打分
def factor_rank(stock,group_by_name,factor_name,factor_rank_name):
    factor=[]
    for i,data in stock.groupby(group_by_name):
        data[factor_rank_name]=data[factor_name].rank(pct=True)
        factor.append(data[factor_rank_name])
    factor=pd.concat(factor,axis=0)
    stock=pd.concat([stock_new,factor],axis=1)
    return stock

In [28]:
stock_new=factor_rank(stock_new,'time','mv','mv_rank')
stock_new=factor_rank(stock_new,'time','return','return_rank')
stock_new=factor_rank(stock_new,'time','bm','bm_rank')

In [29]:
stock_new.head()

,code,name,time,mv,return,volume,bm,mv_rank,return_rank,bm_rank
0,000001.SZ,平安银行,2006-01-13,1.247272e+10,0.043974,54697053.0,0.402075,0.972532,0.654936,0.231760
1,000001.SZ,平安银行,2006-01-20,1.208356e+10,-0.031201,56915328.0,0.415024,0.972950,0.032122,0.247675
2,000001.SZ,平安银行,2006-01-25,1.212247e+10,0.003221,36125086.0,0.413685,0.976232,0.430458,0.257923
3,000001.SZ,平安银行,2006-02-10,1.235597e+10,0.019262,27011132.0,0.405877,0.974569,0.730245,0.248865
4,000001.SZ,平安银行,2006-02-17,1.208356e+10,-0.022047,43678212.0,0.415024,0.972003,0.090989,0.270341


In [30]:
stock_new['score']=0.3*stock_new['mv_rank']+0.2*stock_new['return_rank']+0.5*stock_new['bm_rank']

In [31]:
# 筛选前100
stock=[]
for i,data in stock_new.groupby('time'):
    data=data.sort_values('score').iloc[0:100,:]
    stock.append(data)
stock=pd.concat(stock,axis=0)

In [32]:
stock_new=[]
for i,data in stock.groupby('time'):
    data['weekly_return']=1/100*data['return']
    stock_new.append(data)
stock_new=pd.concat(stock_new,axis=0)

In [33]:
stock_return=stock_new.groupby('time')['weekly_return'].sum()
stock_return.index=stock_return.index.astype('datetime64[ns]')
stock_return

time
2006-01-13    0.018009
2006-01-20    0.017061
2006-01-25   -0.008748
2006-02-10   -0.064638
2006-02-17    0.025261
                ...   
2021-09-24   -0.066916
2021-09-30   -0.048155
2021-10-08   -0.074438
2021-10-15   -0.000229
2021-10-22   -0.047930
Name: weekly_return, Length: 819, dtype: float64

In [38]:
stock_return.describe()[1:3]

mean   -0.034993
std     0.050355
Name: weekly_return, dtype: float64